In [ ]:
import pandas as pd
import numpy as np


# Buy and Hold

In [ ]:
weights_file = '../../../portfolio-mgmt/stats/1d_BAH_0.1_weights.csv'
prices_file = '../../datasets/thesis/1d/all_1d.csv'

In [ ]:
# w = pd.read_csv(weights_file)
# w.index = pd.to_datetime(w['date'])
# w = w.drop('date', axis=1)
# w

In [ ]:
p = pd.read_csv(prices_file, index_col=0)
# p.index = pd.to_datetime(p['date'])
# p = p.drop('date', axis=1)
p = p.pivot(index="date", columns="tic", values="close")
p = p[p.index >= '2022-06-21 23:00:00']
p

In [ ]:
# p0 = p.iloc[24]
# px = p.iloc[24:49]
# p1 = p.iloc[48]
# p0, p1

In [ ]:
fee = 0.001
p0 = p.iloc[0]
amount = 1_000_000
money_each = amount * 0.1
amount_each = (money_each / p0) * (1 - fee)

In [ ]:
money_all = amount_each * p
money_all['sum'] = money_all.sum(axis=1)
money_all['return'] = money_all['sum'].pct_change() + 1
money_all.loc[money_all.iloc[0].name]['return'] = 1 - fee
money_all['equity'] = (money_all['return']).cumprod()
bah_return = money_all[['return', 'equity']]
bah_value = money_all[p.columns]

In [ ]:
bah_return.to_csv('benchmark/1d_BAH_equity.csv')
bah_value.to_csv('benchmark/1d_BAH_value.csv')

# CRP

In [ ]:
crp_weights_file = '../../../portfolio-mgmt/stats/1d_CRP_0.1_weights.csv'
crp_target_file = 'benchmark/1d_CRP_equity.csv'

In [ ]:
w = pd.read_csv(crp_weights_file)
w.index = pd.to_datetime(w['date'])
w = w.drop('date', axis=1)
w = w.drop('CASH', axis=1)

In [ ]:
fee = 0.001
p0 = p.iloc[0]
amount = 1_000_000
money_each = amount * w.iloc[0]
amount_each = (money_each / p0) * (1 - fee)

In [ ]:
money_intraday = amount_each * p.iloc[0:1]
money_total = money_intraday.sum(axis=1)
# last_money_total = money_total.iloc[-1]
returns = money_total.pct_change() + 1
returns.loc[returns.index[0]] = 1 - fee # only for first iteration
amount_previous = amount_each

In [ ]:
returns.to_csv(f'{crp_target_file}', float_format='{:.12f}'.format, index=True, header=True)

In [ ]:
for i, x in enumerate(w.index):
    # print(i, x)
    last_total = money_total.iloc[-1]
    start_idx = 0 + (1 * i)
    end_idx = start_idx + 1
    if start_idx >= p.shape[0]:
        break
    price_rebal = p.iloc[start_idx]
    weight_target = w.iloc[i + 1]
    
    new_total_temp = (price_rebal * amount_previous).sum()
    new_value = new_total_temp * weight_target
    amount_target = new_value  / price_rebal
    amount_change = amount_target - amount_previous
    
    amount_new = amount_target.copy()
    amount_new[amount_change > 0] = np.NaN
    increased_amount = amount_previous + ((amount_change[amount_change > 0] * price_rebal * (1 - fee) * (1 - fee)) / price_rebal)
    amount_new[amount_new.isna()] = increased_amount[amount_new.isna()]
    
    money_intraday = amount_new * p.iloc[start_idx:end_idx]
    money_total = money_intraday.sum(axis=1)
    returns = money_total.pct_change() + 1
    returns.iloc[0] = money_total.iloc[0] / last_total
    returns.to_csv(f'{crp_target_file}', float_format='{:.12f}'.format, mode='a', index=True, header=False)
    
    amount_previous = amount_new
    

In [ ]:
df = pd.read_csv(crp_target_file, index_col=0)
df.columns = ['return']
df['equity'] = df['return'].cumprod()
df.to_csv(f'{crp_target_file}', float_format='{:.12f}'.format, index=True, header=True)

# MPT 

In [ ]:
mpt_weights_file = '../../../portfolio-mgmt/stats/1d_MPT_0.1_weights.csv'
mpt_target_file = 'benchmark/1d_MPT_equity.csv'

In [ ]:
w = pd.read_csv(mpt_weights_file)
w.index = pd.to_datetime(w['date'])
w = w.drop('date', axis=1)
# w = w.drop('CASH', axis=1)

In [ ]:
fee = 0.001
p0 = p.iloc[0]
amount = 1_000_000
money_each = amount * w.iloc[0]
amount_each = (money_each / p0) * (1 - fee)

In [ ]:
money_intraday = amount_each * p.iloc[0:1]
money_total = money_intraday.sum(axis=1)
# last_money_total = money_total.iloc[-1]
returns = money_total.pct_change() + 1
returns.loc[returns.index[0]] = 1 - fee # only for first iteration
amount_previous = amount_each

In [ ]:
returns.to_csv(f'{mpt_target_file}', float_format='{:.12f}'.format, index=True, header=True)

In [ ]:
for i, x in enumerate(w.index):
    # print(i, x)
    last_total = money_total.iloc[-1]
    start_idx = 0 + (1 * i)
    end_idx = start_idx + 1
    if start_idx >= p.shape[0]:
        break
        
    price_rebal = p.iloc[start_idx]
    weight_target = w.iloc[i + 1]
    
    new_total_temp = (price_rebal * amount_previous).sum()
    new_value = new_total_temp * weight_target
    amount_target = new_value  / price_rebal
    amount_change = amount_target - amount_previous
    
    amount_new = amount_target.copy()
    amount_new[amount_change > 0] = np.NaN
    increased_amount = amount_previous + ((amount_change[amount_change > 0] * price_rebal * (1 - fee) * (1 - fee)) / price_rebal)
    amount_new[amount_new.isna()] = increased_amount[amount_new.isna()]
    
    money_intraday = amount_new * p.iloc[start_idx:end_idx]
    money_total = money_intraday.sum(axis=1)
    returns = money_total.pct_change() + 1
    returns.iloc[0] = money_total.iloc[0] / last_total
    returns.to_csv(f'{mpt_target_file}', float_format='{:.12f}'.format, mode='a', index=True, header=False)
    
    amount_previous = amount_new
    

In [ ]:
df = pd.read_csv(mpt_target_file, index_col=0)
df.columns = ['return']
df['equity'] = df['return'].cumprod()
df.to_csv(f'{mpt_target_file}', float_format='{:.12f}'.format, index=True, header=True)

# Generate Combined Results File

In [ ]:
names = ['CRP', 'BAH', 'MPT']
files = ['benchmark/1d_CRP_equity.csv', 'benchmark/1d_BAH_equity.csv', 'benchmark/1d_MPT_equity.csv']

In [ ]:
idx = 0
df = pd.read_csv(files[idx], index_col=0)
current_returns = df[['return']]
current_returns = current_returns.rename(columns={'return': names[idx]})
current_returns.shape

In [ ]:
idx = 1
df2 = pd.read_csv(files[idx], index_col=0)
current_returns2 = df2[['return']]
current_returns2 = current_returns2.rename(columns={'return': names[idx]})
current_returns2.shape

In [ ]:
idx = 2
df3 = pd.read_csv(files[idx], index_col=0)
current_returns3 = df3[['return']]
current_returns3 = current_returns3.rename(columns={'return': names[idx]})
current_returns3.shape

In [ ]:
res = current_returns[['CRP']]
res['BAH'] = current_returns2[['BAH']]
res['MPT'] = current_returns3[['MPT']]

In [ ]:
res.to_csv('benchmark/1d_benchmark_returns.csv')

In [ ]:
current_returns = df[['return']]